# TP SQL Agrégation Informatique

date du document: 23 septembre 2024

Le système utilisé pendant les TME est H2.

- Connectez-vous à l'environnement Jupyterhub de Capsule  (section TMEs Jupyter Notebook)
- Importez le fichier TD_TP_4_ETUDIANTS.ipynb dans votre environnement  (flèche vers le haut en haut à gauche)
- Sélectionnez le Kernel (Python [conda env:pyspark])
- Le schéma correspondant à la base du fichier bd-jo-v2-H2.sql se trouve dans le fichier jo_v2_diagram.pdf


Alternatives si problème d'utilisation de Jupyterhub de Capsule:

- Installer et exécuter le serveur H2 en local (voir le documentation la documentation SGBD H2)
- Télécharger les fichiers du TME dans un répertoire local et lancer jupyter-lab ou jupyter-notebook en local (installer  java si besoin), il n'y a pas de kernel pyspark dans ce cas


In [2]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# executer uniquemement sur jupyterhub Moodle
# pip install pyspark

**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger les pilote PostgreSQL et H2

In [4]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-09-30 18:19:52--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1081604 (1,0M) [application/octet-stream]
Enregistre : ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    ds 0,09s   

2024-09-30 18:19:52 (11,6 MB/s) - ‘postgresql-42.6.0.jar’ enregistré [1081604/1081604]

--2024-09-30 18:19:52--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2543011 (2,4M) [application/octet-stream]
Enregistre : ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  13,8MB/s    ds 0,

## Fonctions utiles

## connect_H2

In [3]:
def connect_H2(db,user,password,port=5435):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [4]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount and cursor.description is not None:
            names = [desc[0] for desc in cursor.description]
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr).replace('\n',' ')))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr).replace('\n',' ')[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

## show table

In [5]:
def show_table(connection,table_name):

    query=f"""
SELECT table_name,
       column_name,
       data_type,
       column_default,
       is_nullable,
       character_maximum_length,
       numeric_precision
FROM information_schema.columns 
where lower(table_name)  = '{table_name}'  
"""
    execute(connection,query)
    

## show_schema

In [6]:
def show_schema(connection):

    print('*********** Tables ************')
    query="""
    select TABLE_NAME
    from INFORMATION_SCHEMA.TABLES 
    where TABLE_SCHEMA = 'public'
    """
    execute(connection,query,show=True)

    print('\n\n*********** Domaines ************')
    query="""
    SELECT domain_name,check_clause 
    FROM information_schema.domain_constraints a, information_schema.check_constraints b
    where a.constraint_name=b.constraint_name
    """
    execute(connection,query)

    print('\n\n*********** Attributs ************')
    query=f"""
    SELECT c.table_name,
           c.column_name,
           c.data_type,
           c.column_default,
           c.is_nullable,
           c.character_maximum_length,
           c.numeric_precision
    FROM INFORMATION_SCHEMA.TABLES t, information_schema.columns c
    where t.table_name=c.table_name
    and t.TABLE_SCHEMA = 'public'
    """
    execute(connection,query)
    

## Démarrer le serveur SQL H2

In [7]:
import psycopg2
import os
import subprocess


local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)

# binaire java : décommenter sur jupyterhub Moodle
# java='/opt/tljh/user/envs/pyspark/bin/java'
java='java'

# chaque utilisateur doit avoir un port différent!
port = 5010

print(f'La commande java est: {java}')
print(f'Le numero du port du serveur H2 est: {port}')
print(f'Les fichiers dans {local_dir}:')
for f in os.listdir(local_dir):
    print(f'   {f}')

La commande java est: java
Le numero du port du serveur H2 est: 5010
Les fichiers dans ./data:
   arbres5010.mv.db
   employes5010.mv.db
   jo5008.mv.db
   jo5010.mv.db
   villes5010.mv.db


In [8]:
output=subprocess.run(f'{java} --version',shell=True, text=True, stdout=subprocess.PIPE).stdout
print(output)

openjdk 11.0.24 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)



In [9]:
# Défnir le port pour le serveur H2
cmd=f"{java} -Dh2.bindAddress=127.0.0.1  -cp  h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort {port}  -baseDir ./data -ifNotExists"
proc=subprocess.Popen(cmd,
                      shell=True,
                      text=True,
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE, 
                      )
try:
    outs, errs = proc.communicate(timeout=1)
    print(errs)
except :
    print("Serveur H2 en marche")

Exception in thread "main" org.h2.jdbc.JdbcSQLNonTransientConnectionException: Exception à l'ouverture du port "5010" (le port est peut-être en cours d'utilisation), cause: "java.net.BindException: Adresse déjà utilisée (Bind failed)"
Exception opening port "5010" (port may be in use), cause: "java.net.BindException: Adresse déjà utilisée (Bind failed)" [90061-214]
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:678)
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:477)
	at org.h2.message.DbException.get(DbException.java:212)
	at org.h2.util.NetUtils.createServerSocketTry(NetUtils.java:211)
	at org.h2.util.NetUtils.createServerSocket(NetUtils.java:177)
	at org.h2.server.pg.PgServer.start(PgServer.java:174)
	at org.h2.tools.Server.start(Server.java:521)
	at org.h2.tools.Server.runTool(Server.java:331)
	at org.h2.tools.Server.main(Server.java:126)
Caused by: java.net.BindException: Adresse déjà utilisée (Bind failed)
	at java.base/java.net.PlainSock

In [10]:
connection =  connect_H2("employes"+f'{port}',"ba","ba",port)

Connection to H2 DB successful


# Manipulation de données : SQL DDL et DML

Les exercices suivantes correspondent aux chapitres correspondants dans les documents
*poly-TD-etudiants-2023.pdf* et *poly-TME-etudiants-2023.pdf*
sur Moodle.

## TD8 CREATION DE SCHEMAS- CONTRAINTES D'INTEGRITE



On considère le schéma Entreprise décrit ci-dessous.

EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)

PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)

EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)

GRILLE_SAL (Profil, salaire)

La clé primaire de chaque relation est soulignée et les attributs des clés étrangères sont suivis d’un
astérisque. Cette base contient des informations sur des employés et sur les projets dans lesquels ils
sont impliqués. DateEnreg dans la table employe donne la date à laquelle l'employé a été enregistré
dans la BD. Ces employés sont embauchés dans un projet sur un profil donné et perçoivent un
salaire en fonction de ce profil. Le chef d'un employé dans la table Employé et le chef d'un projet
dans la table Projet sont des employés. En plus des contraintes de clé et contraintes référentielles
indiquées dans le schéma, on voudrait intégrer les contraintes suivantes :

* Domaines
    - Le numéro de sécurité sociale possède exactement 5 chiffres.
    - Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils peuvent en avoir moins).
    - La ville d’un employé (VillE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et sa longueur ne dépasse pas 9 caractères.
    - Le numéro d’un projet varie entre 5 et 7 chiffres.
    - Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.
    - Le budget est un entier sur 6 chiffres (sans virgule)
    - Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table Employé.

* Clés
    - Il n'y a pas deux employés avec le même nom et le même prénom.

* Not-null
    - Chaque projet doit avoir un responsable.
    - Dans la table Embauche, NumSS; NumProj et Profil représentent un employé, un projet et un profil existant.


#### Donner en SQL les instructions de création des contraintes de domaines indiquées dans l'énoncé.

In [13]:
execute(connection,"drop all objects")

0 rows


<cursor object at 0x754adb552b60; closed: -1>

In [14]:
query="""
/* Le numéro de sécurité sociale possède exactement 5 chiffres.*/
drop domain dnumss if exists;
create domain dnumss as  varchar(5)
"""
execute(connection,query)

0 rows


<cursor object at 0x754adb550f40; closed: -1>

In [15]:
execute(connection,"select * from information_schema.domains")

1 rows
|domain_catalog|domain_schema|domain_name|data_type        |character_maximum_length|character_octet_length|character_set_catalog|character_set_schema|character_set_name|collation_catalog|collation_schema|collation_name|numeric_precision|numeric_precision_radix|numeric_scale|datetime_precision|interval_type|interval_precision|domain_default|maximum_cardinality|dtd_identifier|declared_data_type|declared_numeric_precision|declared_numeric_scale|geometry_type|geometry_srid|domain_on_update|parent_domain_catalog|parent_domain_schema|parent_domain_name|remarks|
|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<cursor object at 0x754adb551120; closed: -1>

In [16]:
query="""
/* Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils
peuvent en avoir moins).*/
drop domain dchaines if exists;
create domain dchaines as  varchar(20);
"""
execute(connection,query)

0 rows


<cursor object at 0x754adb552020; closed: -1>

In [17]:
query="""
/* La ville d’un employé (VilleE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et
sa longueur ne dépasse pas 9 caractères.*/
drop domain dvilles if exists;
CREATE DOMAIN dvilles as  VARCHAR(9)
CHECK VALUE IN ('Paris', 'Lyon', 'Marseille');
"""
execute(connection,query)
show_schema(connection)

0 rows
*********** Tables ************
0 rows


*********** Domaines ************
1 rows
|domain_name|check_clause                          |
|---------------------------------------------------
|dvilles    |VALUE IN('Paris', 'Lyon', 'Marseille')|


*********** Attributs ************
0 rows


In [18]:
query="""
/*Le numéro d’un projet varie entre 5 et 7 chiffres.*/
drop domain dnumproj if exists;
create domain dnumproj as NUMERIC(7)
CHECK (LENGTH(VALUE) >= 5 AND LENGTH(VALUE) <= 7);


/* Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.*/
drop domain dsal if exists;
create domain dsal as NUMERIC(7,2)
CHECK VALUE <= 90000.00;

/*Budget sur 6 chiffres*/
drop domain dbudget if exists;
create domain dbudget as NUMERIC(6,0);

/*Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table
Employé.*/
drop domain ddatenaiss if exists;
create domain ddatenaiss as DATE
CHECK (datediff(year, VALUE, CURRENT_DATE) <= 70);
"""
execute(connection,query)
show_schema(connection)


0 rows
*********** Tables ************
0 rows


*********** Domaines ************
4 rows
|domain_name|check_clause                                               |
|------------------------------------------------------------------------
|dsal       |VALUE <= 90000.00                                          |
|ddatenaiss |DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70                  |
|dvilles    |VALUE IN('Paris', 'Lyon', 'Marseille')                     |
|dnumproj   |(CHAR_LENGTH(VALUE) >= 5)     AND (CHAR_LENGTH(VALUE) <= 7)|


*********** Attributs ************
0 rows


### Donner en SQL les instructions de création du schéma de la base en leur associant les contraintes de clés, référentielles et de domaines indiquées dans l'énoncé.

In [19]:
query="""
/*EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)*/
create table Employe(
    NumSS dnumss PRIMARY KEY,
    NomE TEXT,
    PrenomE TEXT,
    NumChef dnumss,
    VilleE dvilles,
    DateNaiss ddatenaiss,

    FOREIGN KEY (NumChef) REFERENCES Employe(NumSS),
    UNIQUE (NomE, PrenomE)
)
"""
execute(connection,query)


0 rows


<cursor object at 0x754adb551e40; closed: -1>

In [20]:
query="""
/*PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)*/
create table Projet(
    NumProj dnumproj PRIMARY KEY, 
    NomProj TEXT, 
    RespProj dnumss NOT NULL,
    VilleP dvilles,
    Budget dbudget,

    FOREIGN KEY (RespProj) REFERENCES Employe(NumSS)
)
"""
execute(connection,query)




0 rows


<cursor object at 0x754adb5527a0; closed: -1>

In [21]:
query="""
/*GRILLE_SAL (Profil, salaire)*/
create table Grille_sal(
    Profil TEXT PRIMARY KEY,
    Salaire dsal    
)
"""
execute(connection,query)


0 rows


<cursor object at 0x754adb552d40; closed: -1>

In [22]:
query="""
/*EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)*/
create table Embauche(
    NumSS dnumss,
    NumProj dnumproj,
    DateEmb DATE,
    Profil TEXT,

    PRIMARY KEY (NumSS, NumProj),
    FOREIGN KEY (NumSS) REFERENCES Employe(NumSS),
    FOREIGN KEY (NumProj) REFERENCES Projet(NumProj),
    FOREIGN KEY (Profil) REFERENCES Grille_sal(Profil)
)
"""
execute(connection,query)


0 rows


<cursor object at 0x754adb552c50; closed: -1>

3. Un responsable de projet doit habiter la ville du projet dont il est responsable. Donner en
SQL l'instruction permettant d’exprimer cette contrainte en considérant que la table projet a été créée/ Utiliser la commande

      alter table <table> add constraint <nom><expression>

In [23]:
query="""
alter table Projet add constraint RespProjer
check (VilleP = 
            (SELECT VilleE from Employe WHERE Projet.RespProj = Employe.NumSS))
"""
execute(connection,query)


0 rows


<cursor object at 0x754adb5532e0; closed: -1>

Commande utile:
    

In [24]:
query="""
alter table PROJET drop constraint resprojet
"""
execute(connection,query)


The error 'Contrainte "resprojet" non trouvée
Constraint "resprojet" not found; SQL statement:

alter table PROJET drop constraint resproje [90057-214]
DETAIL:  org.h2.jdbc.JdbcSQLSyntaxErrorException: Contrainte "resprojet" non trouvée
Constraint "resprojet" not found; SQL statement:

alter table PROJET drop constraint resproje [90057-214]
' occurred


## TME8 CREATION DE SCHEMAS- CONTRAINTES D'INTEGRITE

Dans ce qui suit, il vous est demandé d'effectuer des insertions de n-uplets dans des tables.

Syntaxe des insertions :

    insert into Table(attr1, attr2, ...) values ('val1', 'val2', ....)
    insert into Table values ('val1', 'val2', ....)
    insert into Table <requête>
    

Syntaxe des suppressions :

    delete from table
    delete from table where <cond>

Syntaxe des mise à jour :

    upate table set attr = valeur where <cond>
    upate table set attr = <requete> where <cond>

In [25]:
connection = connect_H2(f"employes{port}","ba","ba",port)

Connection closed
Connection to H2 DB successful


### Insérez dans chaque table au moins un n-uplet qui vérifie les contraintes d'intégrité.

In [26]:
query="""
/*insert into employe(NUMSS,NOME,PRENOME, VILLEE , DATENAISS ) values (12345, 'Smith', 'John', 'Paris', '1975-08-25' );*/
/*insert into projet values (123456,'Tah le projet de FOU', 12345, 'Paris', 750000);*/
/*insert into grille_sal values('Ingénieur Bomboclaat', 60000.25);*/
/*insert into embauche values (12345, 123456, '2019-01-01', 'Ingénieur Bomboclaat');*/
"""
execute(connection,query)

0 rows


<cursor object at 0x754adb551f30; closed: -1>

### Proposez des insertions qui violent les contraintes d'intégrité définies pour chaque table.

1. Proposer une insertion dans la table Employé qui ne respecte pas la contrainte de clé primaire.

In [27]:
query="""
/*insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
   values (21456, 'DUPONT', 'Marc', 'Paris', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ; */
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
   values (21456, 'GROSCACA', 'Edouard', 'Paris', '1977-05-14')
"""
execute(connection,query)

1 rows


<cursor object at 0x754adb552e30; closed: -1>

2. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de limite d'âge.

In [11]:
query="""
insert into employe 
values (47035, 'GROSPROUT', 'Mireille', 00000, 'Paris', '1840-04-28')
"""
execute(connection,query)

The error 'Vérifiez la violation de contrainte: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"
Check constraint violation: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"; SQL statement:

insert into employe 
values (47035, 'GROSPROUT', 'Mireille', 00000, 'Paris', '1840-04-28' [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"
Check constraint violation: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"; SQL statement:

insert into employe 
values (47035, 'GROSPROUT', 'Mireille', 00000, 'Paris', '1840-04-28' [23513-214]
' occurred


3. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de longueur de
l'attribut NumSS.

In [29]:
query="""
insert into employe 
values (22222222222222, 'DeBZ', 'Chokbarinho', 00000, 'Paris', '2002-01-30')
"""
execute(connection,query)

The error 'Valeur trop longue pour la colonne "numss CHARACTER VARYING(5)": "22222222222222 (64)"
Value too long for column "numss CHARACTER VARYING(5)": "22222222222222 (64)"; SQL statement:

insert into employe 
values (22222222222222, 'DeBZ', 'Chokbarinho', 00000, 'Paris', '2002-01-30' [22001-214]
DETAIL:  org.h2.jdbc.JdbcSQLDataException: Valeur trop longue pour la colonne "numss CHARACTER VARYING(5)": "22222222222222 (64)"
Value too long for column "numss CHARACTER VARYING(5)": "22222222222222 (64)"; SQL statement:

insert into employe 
values (22222222222222, 'DeBZ', 'Chokbarinho', 00000, 'Paris', '2002-01-30' [22001-214]
' occurred


4. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte sur les villes
possibles.

In [30]:
query="""
insert into employe
values (78046, 'Matuidi', 'Blaise', 00000, 'Les Ulis', '1995-06-30')
"""
execute(connection,query)

The error 'Vérifiez la violation de contrainte: "(VALUE IN('Paris', 'Lyon', 'Marseille'))"
Check constraint violation: "(VALUE IN('Paris', 'Lyon', 'Marseille'))"; SQL statement:

insert into employe
values (78046, 'Matuidi', 'Blaise', 00000, 'Les Ulis', '1995-06-30' [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "(VALUE IN('Paris', 'Lyon', 'Marseille'))"
Check constraint violation: "(VALUE IN('Paris', 'Lyon', 'Marseille'))"; SQL statement:

insert into employe
values (78046, 'Matuidi', 'Blaise', 00000, 'Les Ulis', '1995-06-30' [23513-214]
' occurred


6. Proposer une insertion dans la table Grille_SAL qui ne respecte pas la contrainte sur le salaire
qui ne doit pas dépasser 90 000.

In [31]:
query="""
insert into  Grille_sal 
values ('Mesureur de bazouzou', 35000000)
"""
execute(connection,query)

The error 'Valeur trop longue pour la colonne "salaire NUMERIC(7, 2)": "35000000 (32)"
Value too long for column "salaire NUMERIC(7, 2)": "35000000 (32)"; SQL statement:

insert into  Grille_sal 
values ('Mesureur de bazouzou', 35000000 [22001-214]
DETAIL:  org.h2.jdbc.JdbcSQLDataException: Valeur trop longue pour la colonne "salaire NUMERIC(7, 2)": "35000000 (32)"
Value too long for column "salaire NUMERIC(7, 2)": "35000000 (32)"; SQL statement:

insert into  Grille_sal 
values ('Mesureur de bazouzou', 35000000 [22001-214]
' occurred


7. Proposer une insertion dans la table Projet qui ne respecte pas la contrainte référentielle vers Employe.

In [32]:
query="""
insert into Projet values (011456, 'Projet tous les choquer', 80000, 'Lyon', 550420)
"""
cursor=execute(connection,query)

The error 'Intégrité référentielle violation de contrainte: "CONSTRAINT_C5: public.projet FOREIGN KEY(respproj) REFERENCES public.employe(numss) ('80000')"
Referential integrity constraint violation: "CONSTRAINT_C5: public.projet FOREIGN KEY(respproj) REFERENCES public.employe(numss) ('80000')"; SQL statement:

insert into Projet values (011456, 'Projet tous les choquer', 80000, 'Lyon', 550420 [23506-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Intégrité référentielle violation de contrainte: "CONSTRAINT_C5: public.projet FOREIGN KEY(respproj) REFERENCES public.employe(numss) ('80000')"
Referential integrity constraint violation: "CONSTRAINT_C5: public.projet FOREIGN KEY(respproj) REFERENCES public.employe(numss) ('80000')"; SQL statement:

insert into Projet values (011456, 'Projet tous les choquer', 80000, 'Lyon', 550420 [23506-214]
' occurred


8. Proposer une insertion dans la table Embauche qui ne respecte pas une des contraintes
référentielles.

In [33]:
query="""
insert into embauche values (12345, 820652, '2024-01-02', 'Mesureur de zizi')

"""
execute(connection,query)

The error 'Intégrité référentielle violation de contrainte: "CONSTRAINT_2EA: public.embauche FOREIGN KEY(numss) REFERENCES public.employe(numss) ('12345')"
Referential integrity constraint violation: "CONSTRAINT_2EA: public.embauche FOREIGN KEY(numss) REFERENCES public.employe(numss) ('12345')"; SQL statement:

insert into embauche values (12345, 820652, '2024-01-02', 'Mesureur de zizi' [23506-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Intégrité référentielle violation de contrainte: "CONSTRAINT_2EA: public.embauche FOREIGN KEY(numss) REFERENCES public.employe(numss) ('12345')"
Referential integrity constraint violation: "CONSTRAINT_2EA: public.embauche FOREIGN KEY(numss) REFERENCES public.employe(numss) ('12345')"; SQL statement:

insert into embauche values (12345, 820652, '2024-01-02', 'Mesureur de zizi' [23506-214]
' occurred


## TD9 CREATION DES SCHEMAS - MODIFICATION DES DONNEES.

### Villes et Pays

On veut créer un schéma relationnel pour stocker des informations sur des villes et des pays.

#### 1. Traduisez le schéma relationnel suivant en instructions SQL:

    Ville(nom, population, pays*)
    Pays(nom, capitale*)

où pays est une référence vers un pays dans la table Pays et capitale est une référence vers une ville dans la table Ville.

In [30]:
connection = connect_H2(f"villes{port}","ba","ba",port)

Connection closed
Connection to H2 DB successful


In [31]:
query="""
drop all objects;
create table Ville(
    nomV VARCHAR(50) UNIQUE,
    population INT,
    pays VARCHAR(50),

    primary key (nomV, pays)
);
create table Pays(
    nomP VARCHAR(50) PRIMARY KEY,
    capitale VARCHAR(50),

    foreign key (capitale) references Ville(nomV)
);

alter table Ville add constraint ville_ref_pays foreign key (pays) references Pays(nomP)

"""
execute(connection,query)


0 rows


<cursor object at 0x79eea0225e40; closed: -1>

#### Insérez la France avec sa capitale Paris (3 millions d'habitants) dans la base de données.

In [32]:
query="""
    alter table Ville drop constraint ville_ref_pays;
    insert into Ville values ('Paris', 3000000, 'France');
    insert into Pays values ('France', 'Paris');
    alter table Ville add constraint ville_ref_pays foreign key (pays) references Pays(nomP)


"""
cursor=execute(connection,query)

0 rows


In [33]:
query="""
    select * from Ville

"""
cursor=execute(connection,query)

1 rows
|nomv |population|pays  |
|------------------------
|Paris|3000000   |France|


#### Modifiez le schéma de telle manière que la suppression d'un pays déclenche automatiquement la suppression de toutes les villes du pays.
                                             

In [36]:
query="""
/*alter table Ville drop constraint fk_pays;*/
alter table Ville add constraint fk_pays on delete set null;

/*alter table Pays drop constraint fk_pays_capitale;*/
alter table Pays add constraint fk_pays_capitale on delete cascade;
"""
execute(connection,query)



The error 'Erreur de syntaxe dans l'instruction SQL "\000a/*alter table Ville drop constraint fk_pays;*/\000aalter table Ville add constraint fk_pays [*]on delete set null"; attendu "., COMMENT, PRIMARY, UNIQUE, FOREIGN, CHECK"
Syntax error in SQL statement "\000a/*alter table Ville drop constraint fk_pays;*/\000aalter table Ville add constraint fk_pays [*]on delete set null"; expected "., COMMENT, PRIMARY, UNIQUE, FOREIGN, CHECK"; SQL statement:

/*alter table Ville drop constraint fk_pays;*/
alter table Ville add constraint fk_pays on delete set null [42001-214]
DETAIL:  org.h2.jdbc.JdbcSQLSyntaxErrorException: Erreur de syntaxe dans l'instruction SQL "\000a/*alter table Ville drop constraint fk_pays;*/\000aalter table Ville add constraint fk_pays [*]on delete set null"; attendu "., COMMENT, PRIMARY, UNIQUE, FOREIGN, CHECK"
Syntax error in SQL statement "\000a/*alter table Ville drop constraint fk_pays;*/\000aalter table Ville add constraint fk_pays [*]on delete set null"; expected "

#### Effacez les deux tables Ville et Pays du schéma.

In [38]:
query="""




"""
execute(connection,query)


0 rows


<cursor object at 0x754adb5534c0; closed: -1>

### Arbres ordonnées

Voici une table Arbre qui permet de stocker des arbres ordonnés dans une base de données
relationnelle:

In [37]:
connection = connect_H2(f"arbres{port}","ba","ba",port)

Connection closed
Connection to H2 DB successful


In [38]:
query="""
drop table Arbre if exists;
create table Arbre(
    id numeric(10),
    par numeric(10),
    pos numeric(2),
    constraint pk primary key (id),
    constraint fk foreign key (par) references Arbre (id));
"""
execute(connection,query)


0 rows


<cursor object at 0x79eea0225f30; closed: -1>

- id est l'identifiant du nœud,
- par est l'identifiant du parent,
- pos est la position parmi les enfants ;

#### Inserez l'arbre binaire (1(2(4,5),3(6,7(8,9))))) dans la base de données.

In [42]:
query="""
delete from Arbre;
insert into Arbre Values (1,NULL, 0);
    insert into Arbre Values (2, 1, 0);
        insert into Arbre Values (4, 2, 0);
        insert into Arbre Values (5, 2, 1);
    insert into Arbre Values (3, 1, 1);
        insert into Arbre Values (6, 3, 0);
        insert into Arbre Values (7, 3, 1);
            insert into Arbre Values (8, 7, 0);
            insert into Arbre Values (9, 7, 1);

select * from Arbre
"""
execute(connection,query,show=True)


9 rows
|id|par |pos|
|------------
|1 |None|0  |
|2 |1   |0  |
|4 |2   |0  |
|5 |2   |1  |
|3 |1   |1  |
|6 |3   |0  |
|7 |3   |1  |
|8 |7   |0  |
|9 |7   |1  |


<cursor object at 0x79eea02266b0; closed: -1>

#### Comment faut il modifier le schéma pour empêcher qu'un noeud a deux enfants à la même position?

In [42]:
query="""
alter table Arbre add constraint ....
"""
execute(connection,query)



The error 'Erreur de syntaxe dans l'instruction SQL "\000aalter table Arbre add constraint [*]....\000a"; attendu "identifier"
Syntax error in SQL statement "\000aalter table Arbre add constraint [*]....\000a"; expected "identifier"; SQL statement:

alter table Arbre add constraint ....
 [42001-214]
DETAIL:  org.h2.jdbc.JdbcSQLSyntaxErrorException: Erreur de syntaxe dans l'instruction SQL "\000aalter table Arbre add constraint [*]....\000a"; attendu "identifier"
Syntax error in SQL statement "\000aalter table Arbre add constraint [*]....\000a"; expected "identifier"; SQL statement:

alter table Arbre add constraint ....
 [42001-214]
' occurred


#### Effacez le sous-arbre 3(6,7(8,9)) de la base de données.

In [43]:
query="""



"""
execute(connection,query)


0 rows


<cursor object at 0x754adb553790; closed: -1>

#### Est-ce que l'instruction suivante est possible sur l'arbre initial (avant l'effacement) ?

In [44]:
query="""
delete from Arbre where id=3
"""
execute(connection,query)

0 rows


<cursor object at 0x754adb5535b0; closed: -1>

#### Comment peut-on modifier le schéma pour effacer le sous-arbre avec l'instruction précédente ?

In [45]:
query="""



"""
execute(connection,query)

0 rows


<cursor object at 0x754adb553970; closed: -1>

#### Est-ce qu'il est possible de modifier l'identifiant d'un nœud ?

#### Comment peut-on modifier le schéma pour permettre l'instruction précédente ?

In [46]:
query="""



"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x754adb553a60; closed: -1>

#### Quel est le résultat des instructions suivantes (sur l'arbre initial):

In [47]:
query="""
alter table Arbre drop constraint fk ;
alter table Arbre add constraint fk foreign key (par) references Arbre(id)
      on delete set null;
delete from Arbre where id=3;
select * from Arbre
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x754adb553b50; closed: -1>

# Requêtes SQL

In [13]:
connection = connect_H2(f"jo{port}","ba","ba",port)

Connection closed
Connection to H2 DB successful


Télécharger le fichier SQL contenant les données de la base JO.

In [44]:
#NE PAS EXECUTER CA!!!!! 
#ça supprime le fichier bd-ji-v2
#!wget https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=agreg/bd-jo-v2_H2.sql -O bd-jo-v2_H2.sql

In [14]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection, create_schema)
connection.commit()

0 rows


## Exemple

Tous les athlètes

In [15]:
cursor=execute(connection,query="SELECT nomath FROM athlete",show=True)

2431 rows
|nomath                 |
|------------------------
|BJOERNDALEN            |
|BJOERGEN               |
|AN                     |
|PECHSTEIN              |
|WÜST                   |
|SVENDSEN               |
|AMMANN                 |
|KRAMER                 |
|SABLIKOVA              |
|HAMELIN                |
|MORGENSTERN            |
|HELLNER                |
|HOEFL-RIESCH           |
|DOMRACHEVA             |
|COLOGNA                |
|LOCH                   |
|ZHOU                   |
|NYSTAD                 |
|SACHENBACHER-STEHLE    |
|KALLA                  |
|ZAITSEVA               |
|PLYUSHCHENKO           |
|MAZE                   |
|DAVIS                  |
|FOURCADE               |
|OLSSON                 |
|ZOEGGELER              |
|KOWALCZYK              |
|HENKEL                 |
|BERGER                 |
|NORTHUG                |
|DI CENTA               |
|KUZMINA                |
|PARK                   |
|STECHER                |
|RAICH                  |
|U

In [47]:
cursor.query

b'SELECT nomath FROM athlete'

In [48]:
cursor.rowcount

2431

In [49]:
cursor.description

(Column(name='nomath', type_code=1043),)

## TD 5 SQL JOINTURES

### Jointures

Q1. Les athlètes français (nom pays = 'France') (104 lignes).

In [50]:
query="""
select a.nomAth, a.prenomAth
from Pays p, Athlete a
where a.codePays = p.codePays and p.nomP='France'
"""

cursor=execute(connection,query,show=True)

104 rows
|nomath                 |prenomath     |
|---------------------------------------
|FOURCADE               |Martin        |
|LAMY CHAPPUIS          |Jason         |
|VAULTIER               |Pierre        |
|CHAPUIS                |Jean Frederic |
|BRUNET                 |Marie Laure   |
|DORIN HABERT           |Marie         |
|ANTHONIOZ              |Deborah       |
|MISSILLIER             |Steve         |
|BOVOLENTA              |Arnaud        |
|MARTINOD               |Marie         |
|GAILLARD               |Jean Marc     |
|DE LE RUE              |Paul-Henri    |
|PINTURAULT             |Alexis        |
|BEATRIX                |Jean Guillaume|
|DUVILLARD              |Robin         |
|MANIFICAT              |Maurice       |
|PERRILLAT BOITEUX      |Ivan          |
|JOSSERAND              |Marion        |
|RAMOIN                 |Tony          |
|TRESPEUCH              |Chloe         |
|MATTEL                 |Coline        |
|MIDOL                  |Jonathan      |
|ROLLAN

In [51]:
query="""
select nomAth, prenomAth
from Pays natural join Athlete
where nomP='France'
"""

cursor=execute(connection,query,show=True)

104 rows
|nomath                 |prenomath     |
|---------------------------------------
|FOURCADE               |Martin        |
|LAMY CHAPPUIS          |Jason         |
|VAULTIER               |Pierre        |
|CHAPUIS                |Jean Frederic |
|BRUNET                 |Marie Laure   |
|DORIN HABERT           |Marie         |
|ANTHONIOZ              |Deborah       |
|MISSILLIER             |Steve         |
|BOVOLENTA              |Arnaud        |
|MARTINOD               |Marie         |
|GAILLARD               |Jean Marc     |
|DE LE RUE              |Paul-Henri    |
|PINTURAULT             |Alexis        |
|BEATRIX                |Jean Guillaume|
|DUVILLARD              |Robin         |
|MANIFICAT              |Maurice       |
|PERRILLAT BOITEUX      |Ivan          |
|JOSSERAND              |Marion        |
|RAMOIN                 |Tony          |
|TRESPEUCH              |Chloe         |
|MATTEL                 |Coline        |
|MIDOL                  |Jonathan      |
|ROLLAN

Q2. Les épreuves (sport, nom d'épreuve, catégorie) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (98 lignes).

In [52]:
query="""
select distinct e.nomep, e.categorie, s.nomsp
from epreuve e, sport s
where e.sid = s.sid
order by s.nomsp desc, e.nomep desc
"""

cursor=execute(connection,query,show=True)

98 rows
|nomep                     |categorie|nomsp                               |
|--------------------------------------------------------------------------
|Snowboard cross           |Femmes   |Surf des neiges                     |
|Snowboard cross           |Hommes   |Surf des neiges                     |
|Slopestyle                |Femmes   |Surf des neiges                     |
|Slopestyle                |Hommes   |Surf des neiges                     |
|Slalom parallèle          |Femmes   |Surf des neiges                     |
|Slalom parallèle          |Hommes   |Surf des neiges                     |
|Slalom géant parallèle    |Femmes   |Surf des neiges                     |
|Slalom géant parallèle    |Hommes   |Surf des neiges                     |
|Halfpipe                  |Femmes   |Surf des neiges                     |
|Halfpipe                  |Hommes   |Surf des neiges                     |
|Sprint par équipes        |Femmes   |Ski de fond                         |
|Spr

In [53]:
query="""
select nomep, categorie, nomsp
from epreuve natural join sport
order by nomsp desc, nomep desc
"""

cursor=execute(connection,query,show=True)

98 rows
|nomep                     |categorie|nomsp                               |
|--------------------------------------------------------------------------
|Snowboard cross           |Femmes   |Surf des neiges                     |
|Snowboard cross           |Hommes   |Surf des neiges                     |
|Slopestyle                |Femmes   |Surf des neiges                     |
|Slopestyle                |Hommes   |Surf des neiges                     |
|Slalom parallèle          |Femmes   |Surf des neiges                     |
|Slalom parallèle          |Hommes   |Surf des neiges                     |
|Slalom géant parallèle    |Femmes   |Surf des neiges                     |
|Slalom géant parallèle    |Hommes   |Surf des neiges                     |
|Halfpipe                  |Femmes   |Surf des neiges                     |
|Halfpipe                  |Hommes   |Surf des neiges                     |
|Sprint par équipes        |Femmes   |Ski de fond                         |
|Spr

Q3. Les athlètes ayant participé à une équipe de France (30 lignes).

In [82]:
query="""
select distinct nomAth, prenomAth, dateNaissance, a.codePays
from Athlete as a
natural join Pays p
natural join Equipe e
where nomP = 'France'
and aid in 
    (Select aid 
    from AthletesEquipe)
"""

cursor=execute(connection,query,show=True)

30 rows
|nomath                 |prenomath     |datenaissance|codepays|
|--------------------------------------------------------------
|AYMONIER               |Celia         |None         |FRA     |
|Alexis GODEFROY        |Thibault      |None         |FRA     |
|BAILLARD               |Jeremy        |None         |FRA     |
|BEATRIX                |Jean Guillaume|1988-03-24   |FRA     |
|BESCOND                |Anais         |None         |FRA     |
|BOEUF                  |Alexis        |None         |FRA     |
|BOURZAT                |Fabian        |None         |FRA     |
|BOUTHERIN              |Jeremie       |None         |FRA     |
|BRAUD                  |Francois      |None         |FRA     |
|CARRON                 |Pernelle      |None         |FRA     |
|COSTERG                |Loic          |None         |FRA     |
|DESTHIEUX              |Simon         |None         |FRA     |
|DORIN HABERT           |Marie         |1986-06-19   |FRA     |
|DUVILLARD              |Robin  

Q4. Les épreuves (sport, épreuve, catégorie) auxquelles participent des équipes (25 lignes).

In [93]:
query="""

select distinct s.nomSp, ep.nomEp, ep.categorie
from RangEquipe req
natural join Epreuve ep
natural join Sport s 

"""

cursor=execute(connection,query,show=True)

25 rows
|nomsp                               |nomep                   |categorie|
|------------------------------------------------------------------------
|Biathlon                            |Relais mix              |Mixte    |
|Biathlon                            |relais 4x6km            |Femmes   |
|Biathlon                            |relais 4x7,5km          |Hommes   |
|Bobsleigh                           |bob à deux              |Femmes   |
|Bobsleigh                           |bob à deux              |Hommes   |
|Bobsleigh                           |bob à quatres           |Hommes   |
|Combiné nordique                    |Par équipes             |Hommes   |
|Curling                             |curling                 |Femmes   |
|Curling                             |curling                 |Hommes   |
|Hockey sur glace                    |hockey sur glace        |Femmes   |
|Hockey sur glace                    |hockey sur glace        |Hommes   |
|Luge                         

Q5. Le pays qui a gagné, en équipe, la médaille d'or dans l'épreuve de la catégorie 'Femmes' intitulée 'relais 4x6km' du sport 'Biathlon' (Ukraine).

In [241]:
query="""
Select nomP, nomEp, rang
from pays
natural join Equipe
natural join RangEquipe
natural join Epreuve
natural join Sport
where categorie = 'Femmes' and nomep = 'relais 4x6km' and nomSp = 'Biathlon'
order by rang

"""

cursor=execute(connection,query,show=True)

16 rows
|nomp                         |nomep       |rang|
|------------------------------------------------
|Ukraine                      |relais 4x6km|1   |
|Russie                       |relais 4x6km|2   |
|Norvège                      |relais 4x6km|3   |
|République tchèque           |relais 4x6km|4   |
|Biélorussie                  |relais 4x6km|5   |
|Italie                       |relais 4x6km|6   |
|États-Unis                   |relais 4x6km|7   |
|Canada                       |relais 4x6km|8   |
|Suisse                       |relais 4x6km|9   |
|Pologne                      |relais 4x6km|10  |
|Allemagne                    |relais 4x6km|11  |
|Kazakhstan                   |relais 4x6km|12  |
|Japon                        |relais 4x6km|13  |
|Slovaquie                    |relais 4x6km|14  |
|République populaire de Chine|relais 4x6km|15  |
|Estonie                      |relais 4x6km|16  |


Q6. Les athlètes ayant participé à au moins une épreuve individuelle et au moins une par équipe (372 lignes).

In [114]:
query="""
select distinct nomAth, prenomAth
from Athlete a
natural join RangIndividuel rInd
natural join AthletesEquipe ae
natural join Equipe 

"""

cursor=execute(connection,query,show=True)

372 rows
|nomath             |prenomath        |
|--------------------------------------
|ABRAMENKO          |Evgeny           |
|AHONEN             |Janne            |
|ALEXANDER          |Nicholas         |
|AN                 |Victor           |
|ANEV               |Krasimir         |
|ANGERER            |Tobias           |
|ANTOLEC            |Jan              |
|ANTSYBOR           |Maryna           |
|ANTYPENKO          |Tetyana          |
|ARWIDSON           |Tobias           |
|AYMONIER           |Celia            |
|BABIKOV            |Ivan             |
|BACHLEDA-CURUS     |Katarzyna        |
|BAILEY             |Lowell           |
|BARDAL             |Anders           |
|BAUER              |Armin            |
|BAUER              |Klemen           |
|BAUER              |Lukas            |
|BAUMANN            |Jonas            |
|BEATRIX            |Jean Guillaume   |
|BERGER             |Tora             |
|BERGMAN            |Carl Johan       |
|BESCOND            |Anais     

Q7. Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [124]:
query="""
Select nomAth
from Athlete
group by nomAth
having count(nomAth) >= 2

"""

cursor=execute(connection,query,show=True)

141 rows
|nomath         |
|----------------
|ABRAMENKO      |
|ANDERSON       |
|BAUER          |
|BAUMANN        |
|BECKERT        |
|BJORNSEN       |
|BOE            |
|BOWMAN         |
|BRAATEN        |
|BROWN          |
|BROZ           |
|Benz           |
|CALDWELL       |
|CAVIEZEL       |
|CHOI           |
|COLOGNA        |
|COOK           |
|COSTA          |
|CRAWFORD       |
|CURRIER        |
|DAVIS          |
|DE JONG        |
|DUFOUR-LAPOINTE|
|DUKURS         |
|DVORAK         |
|EDER           |
|EGGER          |
|Ericsson       |
|FAIVRE         |
|FANCHINI       |
|FERNANDEZ      |
|FISCHER        |
|FISCHNALLER    |
|FLETCHER       |
|FONTANA        |
|FOURCADE       |
|GAGNON         |
|GAILLARD       |
|GASPARIN       |
|GASSER         |
|GISIN          |
|GOLD           |
|GREGG          |
|GRUBER         |
|GUTIERREZ      |
|HALLER         |
|HAMELIN        |
|HAMILTON       |
|HANSEN         |
|HARVEY         |
|HERNANDEZ      |
|HOFER          |
|INNERHOFER     |
|

Q8. Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [126]:
query="""
Select nomAth, prenomAth
from Athlete a
natural join RangIndividuel rInd
group by rInd.aid
having count(rInd.aid) >= 2

"""

cursor=execute(connection,query,show=True)

706 rows
|nomath             |prenomath        |
|--------------------------------------
|BJOERNDALEN        |Ole Einar        |
|BJOERGEN           |Marit            |
|AN                 |Victor           |
|PECHSTEIN          |Claudia          |
|WÜST               |Ireen            |
|SVENDSEN           |Emil Hegle       |
|AMMANN             |Simon            |
|KRAMER             |Sven             |
|SABLIKOVA          |Martina          |
|MORGENSTERN        |Thomas           |
|HELLNER            |Marcus           |
|HOEFL-RIESCH       |Maria            |
|DOMRACHEVA         |Darya            |
|COLOGNA            |Dario            |
|SACHENBACHER-STEHLE|Evi              |
|KALLA              |Charlotte        |
|ZAITSEVA           |Olga             |
|MAZE               |Tina             |
|DAVIS              |Shani            |
|FOURCADE           |Martin           |
|OLSSON             |Johan            |
|KOWALCZYK          |Justyna          |
|HENKEL             |Andrea    

Q9. Les athlètes ayant gagné une médaille dans une épreuve individuelle, mais ayant été disqualifiés
dans une autre (14 lignes).

In [140]:
query="""
Select distinct nomAth, prenomAth
from Athlete a
where   exists (Select * from RangIndividuel rind where (rind.aid = a.aid) and rind.rang <= 3)
and     exists (Select * from RangIndividuel rind where (rind.aid = a.aid) and rind.rang is NULL)

"""

cursor=execute(connection,query,show=True)

14 rows
|nomath      |prenomath|
|-----------------------
|FENNINGER   |Anna     |
|GISIN       |Dominique|
|GUT         |Lara     |
|HOEFL-RIESCH|Maria    |
|INNERHOFER  |Christof |
|JANSRUD     |Kjetil   |
|JIANROU     |Li       |
|KOWALCZYK   |Justyna  |
|LIGETY      |Ted      |
|MANCUSO     |Julia    |
|MAYER       |Matthias |
|MISSILLIER  |Steve    |
|PINTURAULT  |Alexis   |
|WEIBRECHT   |Andrew   |


Q10. Les athlètes ayant gagné (au moins) une médaille dans une épreuve individuelle (182 lignes).

In [143]:
query="""
select prenomAth, nomAth
from Athlete a 
where exists (Select * from RangIndividuel rind where (rind.aid = a.aid) and rang <= 3)

"""

cursor=execute(connection,query,show=True)

182 rows
|prenomath       |nomath         |
|---------------------------------
|Ole Einar       |BJOERNDALEN    |
|Marit           |BJOERGEN       |
|Victor          |AN             |
|Ireen           |WÜST           |
|Emil Hegle      |SVENDSEN       |
|Sven            |KRAMER         |
|Martina         |SABLIKOVA      |
|Charles         |HAMELIN        |
|Marcus          |HELLNER        |
|Maria           |HOEFL-RIESCH   |
|Darya           |DOMRACHEVA     |
|Dario           |COLOGNA        |
|Felix           |LOCH           |
|Yang            |ZHOU           |
|Charlotte       |KALLA          |
|Tina            |MAZE           |
|Martin          |FOURCADE       |
|Johan           |OLSSON         |
|Armin           |ZOEGGELER      |
|Justyna         |KOWALCZYK      |
|Tora            |BERGER         |
|Seung-Hi        |PARK           |
|Daniel          |RICHARDSSON    |
|Natalie         |GEISENBERGER   |
|Alexandre       |BILODEAU       |
|Ted             |LIGETY         |
|Lee       

Q11.  Les noms de pays qui n’ont qu’une seule équipe (9 lignes).

In [165]:
query="""
Select distinct nomP, p.codePays
from Pays p
where (Select count (*) from Equipe e where p.codePays = e.codePays) = 1

"""

cursor=execute(connection,query,show=True)

9 rows
|nomp       |codepays|
|---------------------
|Azerbaïdjan|AZE     |
|Belgique   |BEL     |
|Bulgarie   |BUL     |
|Espagne    |ESP     |
|Jamaïque   |JAM     |
|Lituanie   |LTU     |
|Monaco     |MON     |
|Serbie     |SRB     |
|Turquie    |TUR     |


Q12. Les athlètes ayant participé à exactement 2 épreuves individuelles (402 lignes).

In [177]:
query = """
Select nomAth, prenomAth
from RangIndividuel rind
natural join Athlete
group by rind.aid
having count(rind.aid) = 2

"""
cursor=execute(connection,query,show=True)

402 rows
|nomath           |prenomath        |
|------------------------------------
|AMMANN           |Simon            |
|KRAMER           |Sven             |
|SABLIKOVA        |Martina          |
|MORGENSTERN      |Thomas           |
|OLSSON           |Johan            |
|NORTHUG          |Petter Jr.       |
|PARK             |Seung-Hi         |
|STECHER          |Mario            |
|RAICH            |Benjamin         |
|SANG-HWA         |Lee              |
|STOCH            |Kamil            |
|TER MORS         |Jorien           |
|WILD             |Vic              |
|MORRISON         |Denny            |
|MOAN             |Magnus Hovdal    |
|HAAG             |Anna             |
|BECKERT          |Stephanie        |
|SCHLIERENZAUER   |Gregor           |
|FRENZEL          |Eric             |
|BLOKHUIJSEN      |Jan              |
|SHIM             |Suk-Hee          |
|DEMONG           |Bill             |
|LEGKOV           |Alexander        |
|BRIGHT           |Torah            |
|VE

## TME5 SQL JOINTURES / EXISTS

Q1. Les sports auxquels LESSER Erik a participé en supposant qu'il a participé aux épreuves
individuelles (2 lignes).

In [191]:
query="""
Select distinct nomSp
from Sport s
natural join Epreuve ep
natural join RangIndividuel
natural join Athlete a
where nomAth = 'LESSER' and prenomAth = 'Erik'


"""

cursor=execute(connection,query,show=True)

2 rows
|nomsp      |
|------------
|Biathlon   |
|Ski de fond|


Q2. Les athlètes ayant participé aux épreuves individuelles de (au moins) deux sports différents (2 lignes). // A priori les 4 lignes sont correctes ????

In [186]:
query="""
select nomAth, prenomAth
from Athlete a
where (
    Select count (distinct nomSp) 
    from Sport
    natural join Epreuve
    natural join RangIndividuel rind
    where rind.aid = a.aid  
) >= 2

"""

cursor=execute(connection,query,show=True)

4 rows
|nomath  |prenomath|
|-------------------
|TER MORS|Jorien   |
|LESSER  |Erik     |
|PEIFFER |Arnd     |
|PETROVIC|Milanko  |


Q3. Les dates de début et de fin des Jeux Olympique 2014.

In [194]:
query="""
Select min(dateDebut), max(dateFin) 
from Epreuve

"""

cursor=execute(connection,query,show=True)

1 rows
|?column?  |?column?  |
|----------------------
|2014-02-06|2014-02-23|


Q4. Les pays qui ont des participants de moins de 18 ans ou de plus que 40 au 24-02-2016
(Attention : il faut éliminer les athlètes dont on ne connaît pas la date de naissance) (9 ou 7 lignes).

In [45]:
query="""

/*
select nomP
from Athlete a 
natural join Pays
where DATEDIFF(YEAR, a.dateNaissance, '2016-02-24') < 18
or DATEDIFF(YEAR, a.dateNaissance, '2016-02-24') > 40
*/


select *
from Athlete a 
where DATEDIFF(YEAR, a.dateNaissance, '2016-02-24') < 18
or DATEDIFF(YEAR, a.dateNaissance, '2016-02-24') > 40


/*
select * 
from Athlete 
order by dateNaissance asc
*/

"""

cursor=execute(connection,query,show=True)

8 rows
|aid|nomath     |prenomath|datenaissance|codepays|
|-------------------------------------------------
|1  |BJOERNDALEN|Ole Einar|1974-01-27   |NOR     |
|4  |PECHSTEIN  |Claudia  |1972-02-22   |GER     |
|27 |ZOEGGELER  |Armin    |1974-01-04   |ITA     |
|32 |DI CENTA   |Giorgio  |1972-07-10   |ITA     |
|99 |ANDERSON   |Jasey Jay|1975-04-13   |CAN     |
|166|DEMTSCHENKO|Albert   |1971-11-27   |RUS     |
|171|KASAI      |Noriaki  |1972-06-06   |JPN     |
|215|TABATA     |Maki     |1974-11-09   |JPN     |


### NOT EXISTS / NOT IN

Q5. Les équipes n'ayant pas d'athlète (la base de données ne contient pas l'information sur les participants) (13 lignes)

In [44]:
query="""

select distinct e.eqid, codepays 
from Equipe e
where not exists (
    Select *
    from AthletesEquipe ae
    where e.eqid = ae.eqid)

"""

cursor=execute(connection,query,show=True)

13 rows
|eqid|codepays|
|--------------
|159 |SWE     |
|160 |FIN     |
|161 |RUS     |
|162 |GER     |
|163 |JPN     |
|167 |USA     |
|229 |ITA     |
|230 |JPN     |
|231 |FRA     |
|232 |CHN     |
|233 |GER     |
|234 |UKR     |
|235 |GBR     |


Q6. Les équipes qui ont exactement 2 athlètes. Retourner l'équipe en question avec ses deux seuls athlètes (114 lignes).


In [55]:
query="""
    Select ae1.eqid, a1.nomAth, a2.nomAth
    from AthletesEquipe ae1, AthletesEquipe ae2, Athlete a1, Athlete a2

    where 
        a1.aid > a2.aid
    and ae1.eqid = ae2.eqid
    
    and ae1.aid = a1.aid
    and ae2.aid = a2.aid

    and not exists (
        select * from AthletesEquipe ae3
        where ae3.eqid = ae1.eqid
        and   ae3.aid <> ae2.aid 
        and   ae3.aid <> ae1.aid
    )

"""

cursor=execute(connection,query,show=True)

114 rows
|eqid|nomath         |nomath         |
|-------------------------------------
|52  |HUMPHRIES      |MOYSE          |
|53  |MEYERS         |WILLIAMS       |
|54  |EVANS          |GREUBEL        |
|55  |VIS            |KAMPHUIS       |
|56  |FRITZ          |PROKOFF        |
|57  |Emilie MARIEN  |WILLEMSEN      |
|58  |SENKEL         |MARTINI        |
|59  |MAYER          |MEYER          |
|60  |STULNEVA       |UDOBKINA       |
|61  |SCHNEIDER      |SCHNEIDERHEINZE|
|62  |FENLATOR       |JONES          |
|63  |WILSON         |WALKER         |
|64  |VALOIS         |CIOCHETTI      |
|65  |RADJENOVIC     |RAWLINSON      |
|66  |TUECHI         |HENGSTER       |
|67  |PALEEVA        |SERGEEVA       |
|68  |CONSTANTIN     |GRECU          |
|69  |SHIN           |KIM            |
|70  |Mayara DA SILVA|SANTOS         |
|71  |VOEVODA        |ZUBKOV         |
|72  |BAUMANN        |HEFTI          |
|73  |LANGTON        |HOLCOMB        |
|74  |KASJANOV       |BELUGIN        |
|75  |MELBARDIS 

Q7. Les sports qui n'ont pas d'épreuves de catégorie Mixte (12 lignes).

In [58]:
query="""

    select s.nomSp
    from Sport s

    where not exists (
        select * from Epreuve ep
        where ep.sid = s.sid
        and ep.categorie = 'Mixte' 
    )


"""

cursor=execute(connection,query,show=True)

12 rows
|nomsp                               |
|-------------------------------------
|Bobsleigh                           |
|Combiné nordique                    |
|Curling                             |
|Hockey sur glace                    |
|Patinage de vitesse                 |
|Patinage de vitesse sur piste courte|
|Saut à ski                          |
|Skeleton                            |
|Ski acrobatique                     |
|Ski alpin                           |
|Ski de fond                         |
|Surf des neiges                     |


Q8. Les athlètes qui ont gagné une médaille d’or (au moins) mais pas de médaille d’argent ni de bronze (53 lignes)

In [62]:
query="""

    Select a.nomAth, a.prenomAth
    from Athlete a 

    where a.aid in (
        select rind.aid 
        from RangIndividuel rind
        where rang = 1    
    )

    and not a.aid in (
        select rind.aid 
        from RangIndividuel rind
        where rang = 2
        or rang = 3
    )

"""

cursor=execute(connection,query,show=True)

53 rows
|nomath         |prenomath       |
|---------------------------------
|BJOERNDALEN    |Ole Einar       |
|BJOERGEN       |Marit           |
|SVENDSEN       |Emil Hegle      |
|HAMELIN        |Charles         |
|DOMRACHEVA     |Darya           |
|COLOGNA        |Dario           |
|LOCH           |Felix           |
|ZHOU           |Yang            |
|MAZE           |Tina            |
|KOWALCZYK      |Justyna         |
|GEISENBERGER   |Natalie         |
|BILODEAU       |Alexandre       |
|LIGETY         |Ted             |
|SANG-HWA       |Lee             |
|STOCH          |Kamil           |
|TER MORS       |Jorien          |
|GRAABAK        |Joergen         |
|WILD           |Vic             |
|FRENZEL        |Eric            |
|LEGKOV         |Alexander       |
|BRODKA         |Zbigniew        |
|TSUPER         |Alla            |
|MATT           |Mario           |
|VAULTIER       |Pierre          |
|GROOTHUIS      |Stefan          |
|HATTESTAD      |Ola Vigen       |
|KUSHNIR    

Q9. Les athlète(s) qui ont fini dernier d’une épreuve individuelle. Indiquer leur nom, prenom, le nom
du sport et de l’épreuve et leur rang. Attention : il faut filtrer les athlètes disqualifiés (73 lignes).

In [68]:
query="""

    Select a.nomAth, a.prenomAth, sp.nomSp, ep.nomEp, r.rang
    from Athlete a

    natural join rangIndividuel r
    natural join Epreuve ep
    natural join Sport sp

    where r.rang = (
        select max(r2.rang)
        from RangIndividuel r2
        where r2.epid = ep.epid
    )

"""

cursor=execute(connection,query,show=True)

73 rows
|nomath       |prenomath      |nomsp                               |nomep                     |rang|
|---------------------------------------------------------------------------------------------------
|DUBAREZAVA   |Nastassia      |Biathlon                            |10km poursuite            |56  |
|HILDEBRAND   |Franziska      |Biathlon                            |12,5km départ groupé      |28  |
|DUBAREZAVA   |Nastassia      |Biathlon                            |15km                      |56  |
|DUBAREZAVA   |Nastassia      |Biathlon                            |7,5km                     |56  |
|KENESHEV     |Dias           |Biathlon                            |10km                      |87  |
|STEPTSENKO   |Danil          |Biathlon                            |12,5km poursuite          |59  |
|SUMANN       |Christoph      |Biathlon                            |15km départ groupé        |27  |
|GOMBOS       |Karoly         |Biathlon                            |20km           

## TD6 REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET A

Q1. Les athlètes ayant participé à (au moins) une épreuve individuelle et (au moins) une épreuve
par équipe. Exprimer la requête en utilisant ANY (au lieu de IN; voir requête 2.6 dans TD5).
Résultat : (372 lignes)

In [70]:
query="""

    Select a.nomAth, a.PrenomAth
    from Athlete a

    where a.aid = any (
        select r.aid 
        from RangIndividuel r
    )

    and a.aid = any (
        select ae.aid
        from AthletesEquipe ae
        where exists( Select * from RangEquipe re where re.eqid = ae.eqid )
    )


"""
execute(connection,query,show=True)

372 rows
|nomath             |prenomath        |
|--------------------------------------
|BJOERNDALEN        |Ole Einar        |
|BJOERGEN           |Marit            |
|AN                 |Victor           |
|WÜST               |Ireen            |
|SVENDSEN           |Emil Hegle       |
|KRAMER             |Sven             |
|MORGENSTERN        |Thomas           |
|HELLNER            |Marcus           |
|DOMRACHEVA         |Darya            |
|COLOGNA            |Dario            |
|LOCH               |Felix            |
|ZHOU               |Yang             |
|NYSTAD             |Claudia          |
|SACHENBACHER-STEHLE|Evi              |
|KALLA              |Charlotte        |
|ZAITSEVA           |Olga             |
|FOURCADE           |Martin           |
|OLSSON             |Johan            |
|KOWALCZYK          |Justyna          |
|HENKEL             |Andrea           |
|BERGER             |Tora             |
|NORTHUG            |Petter Jr.       |
|DI CENTA           |Giorgio   

<cursor object at 0x70f6e3542e30; closed: -1>

2. a) Les pays ayant au moins un athlète (requête équivalente: les pays ayant participé aux JO).
Résultat : (88 lignes)

In [71]:
query="""

    Select p.nomP
    from Pays p

    where exists ( Select * from Athlete a where a.codePays = p.codePays )

"""
execute(connection,query,show=True)

88 rows
|nomp                                 |
|--------------------------------------
|Albanie                              |
|Andorre                              |
|Argentine                            |
|Arménie                              |
|Australie                            |
|Autriche                             |
|Azerbaïdjan                          |
|Belgique                             |
|Bermudes                             |
|Bosnie-Herzégovine                   |
|Biélorussie                          |
|Brésil                               |
|Bulgarie                             |
|Canada                               |
|Îles Caïmans                         |
|Chili                                |
|République populaire de Chine        |
|Croatie                              |
|Chypre                               |
|République tchèque                   |
|Danemark                             |
|Dominique                            |
|Espagne                        

<cursor object at 0x70f6e35432e0; closed: -1>

b) Les pays ayant exactement un seul athlète
Résultat : (18 lignes)

In [75]:
query="""

    Select p.nomP
    from Pays p, Athlete a 

    where p.codePays = a.codePays
    and not exists (
        select * from Athlete a2
        where a.aid <> a2.aid
        and  a.codePays = a2.codePays
    )

"""
execute(connection,query,show=True)


18 rows
|nomp                       |
|----------------------------
|Bermudes                   |
|Îles Caïmans               |
|Hong Kong                  |
|Îles Vierges des États-Unis|
|Îles Vierges britanniques  |
|Kirghizistan               |
|Luxembourg                 |
|Mexique                    |
|Malte                      |
|Népal                      |
|Pakistan                   |
|Paraguay                   |
|Philippines                |
|Tonga                      |
|Tadjikistan                |
|Timor oriental             |
|Venezuela                  |
|Zimbabwe                   |


<cursor object at 0x70f6e3543100; closed: -1>

3. a) Les athlètes qui n'ont jamais été disqualifiés aux épreuves individuelles.
Résultat : (2194 lignes)

In [81]:
query="""

    Select a.nomAth, a.prenomAth
    from Athlete a

    where not exists ( select * from RangIndividuel r where r.aid = a.aid and r.rang is null)


"""
execute(connection,query,show=True)

2194 rows
|nomath                 |prenomath        |
|------------------------------------------
|BJOERNDALEN            |Ole Einar        |
|BJOERGEN               |Marit            |
|AN                     |Victor           |
|PECHSTEIN              |Claudia          |
|WÜST                   |Ireen            |
|SVENDSEN               |Emil Hegle       |
|AMMANN                 |Simon            |
|KRAMER                 |Sven             |
|SABLIKOVA              |Martina          |
|HAMELIN                |Charles          |
|MORGENSTERN            |Thomas           |
|HELLNER                |Marcus           |
|DOMRACHEVA             |Darya            |
|COLOGNA                |Dario            |
|LOCH                   |Felix            |
|ZHOU                   |Yang             |
|NYSTAD                 |Claudia          |
|KALLA                  |Charlotte        |
|ZAITSEVA               |Olga             |
|PLYUSHCHENKO           |Evgeny           |
|MAZE                 

<cursor object at 0x70f6e3543e20; closed: -1>

b) Les pays qui n'ont pas eu d'athlète disqualifié aux épreuves individuelles.
Résultat : (143 lignes)

In [85]:
query="""
    
    Select distinct nomP
    from Pays

    natural join Athlete a 
    a.aid not in ( Select aid from RangIndividuel where rang is null )

"""
execute(connection,query,show=True)

80 rows
|nomp                                 |
|--------------------------------------
|Allemagne                            |
|Andorre                              |
|Argentine                            |
|Arménie                              |
|Australie                            |
|Autriche                             |
|Azerbaïdjan                          |
|Belgique                             |
|Bermudes                             |
|Biélorussie                          |
|Bosnie-Herzégovine                   |
|Brésil                               |
|Bulgarie                             |
|Canada                               |
|Chili                                |
|Corée du Sud                         |
|Croatie                              |
|Danemark                             |
|Dominique                            |
|Espagne                              |
|Estonie                              |
|Ex-république yougoslave de Macédoine|
|Finlande                       

<cursor object at 0x70f6e0140400; closed: -1>

c) Les pays qui ont participé aux JO et qui n'ont pas eu d'athlètes disqualifiés aux épreuves
individuelles.
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

d) Les pays qui ont participé aux JO et qui n'ont pas eu des athlètes disqualifiés ni en
individuel ni par équipe
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

4. a) Les athlètes n'ayant pas gagné de médaille ni en individuel ni en équipe
Résultat : (1921 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

b) Les pays ayant participé aux JO et n'ayant pas gagné de médaille ni en individuel ni en
équipe.
Résultat : (63 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)


5. L'(es) épreuve(s) avec la plus grande durée.
Résultat : Hockey sur glace, Hockey sur glace, Hommes, 16 jours

In [ ]:
query="""


"""
execute(connection,query,show=True)

6. Les athlètes ayant gagné une médaille à toutes les épreuve individuelles auxquelles ils ont
participé. Résultat : (109 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## TME6 REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY

### ANY / ALL / IN / EXISTS

Exprimer les requêtes suivantes en SQL :

1. Les sports dont toutes les épreuves ont duré un seul jour.
Résultat : Ski de fond, Ski alpin, Biathlon

In [ ]:
query="""


"""
execute(connection,query,show=True)

2. Les sports qui n'ont pas d'épreuves de categorie Mixte.
Résultat : (12 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

3. Les équipes dont aucun athlète n'a gagné de médaille aux épreuves individuelles.
Attention : il y a des équipes sans athlètes.
Résultat : (265 lignes avec les équipes sans athlètes - 252 lignes sans les équipes sans
athlètes)

Avec les équipes sans athlètes :

In [ ]:
query="""


"""
execute(connection,query,show=True)

Sans les équipes sans athlètes :

In [ ]:
query="""


"""
execute(connection,query,show=True)

4. La nationalité de l'athlète le/la plus jeune. Attention : il y a des athlètes dont on ne connaît
pas la date de naissance.
Résultat : ('29/11/1998','JPN')

In [ ]:
query="""


"""
execute(connection,query,show=True)

5. Le plus jeune athlète de chaque pays.
Résultat : (26 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## TME7 REQUÊTES D'AGRÉGATION ET DIVISION

On reprend le schéma schéma « Jeux Olympiques d'hiver 2014 ».

PAYS (CODEPAYS, NOMP)

SPORT (SID , NOM SP)

EPREUVE (EPID, SID*, NOMEP, CATEGORIE, DATEDEBUT, DATEFIN)

ATHLETE (AID, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)

EQUIPE (EQID, CODE PAYS*)

ATHLETESEQUIPE (EQID*, AID*)

RANGINDIVIDUEL (EPID*, AID*, RANG)

RANGEQUIPE (EPID*, EQID*, RANG)

### Fonctions d’agrégation « COUNT, SUM, AVG, MIN, MAX »

1. Le nombre d’athlètes.
Résultat (1 ligne) : 2431

In [ ]:
query="""


"""
execute(connection,query,show=True)

2. Le nombre d’athlètes ayant participé à au moins une épreuve en individuel.
Résultat (1 ligne) : 1558

In [ ]:
query="""


"""
execute(connection,query,show=True)

3. L'âge moyen des sportifs dont le code pays est 'FRA' (France) au 06/02/2014.
Résultat (1 ligne) : 26,8
Aide :
– utilisez round(valeur,nb) pour garder seulement nb décimales à valeur
– sous Oracle, utilisez: to_date('06/02/2014','dd/mm/YYYY')

In [ ]:
query="""


"""
execute(connection,query,show=True)

4. La durée moyenne, minimale et maximale des épreuves.
Résultat (1 ligne) : « Durée moyenne = 1,98 min = 1 max = 16 »
Aide : utilisez l’opérateur de concaténation ||
Attention : entre le 10/01/2014 et le 13/01/2014, il y a une durée de 4 jours (et non pas 3).

In [ ]:
query="""


"""
execute(connection,query,show=True)

5. Le nombre moyen d'athlètes par pays, c'est-à-dire le nombre d'athlètes divisé par le nombre
de pays (ayant au moins un athlète). Résultat (1 ligne) : 27,625

In [ ]:
query="""


"""
execute(connection,query,show=True)

Partitionnement « group by »

6. Pour chaque pays, le nom du pays et le nombre d’athlètes, ordonner par nombre d’athlètes
croissant.
Résultats (88 lignes) : (PAK,1) ; (HKG, 1) ; ... ; (USA, 196) ; (CAN,221)

In [ ]:
query="""


"""
execute(connection,query,show=True)

7. Le nombre moyen d'athlètes par pays (avec group by). Aide : compter le nombre
d’athlètes dans chaque pays (ayant au moins un athlète), puis faire la moyenne.
Résultat (1 ligne) : 27,625

In [ ]:
query="""


"""
execute(connection,query,show=True)

8. Pour chaque équipe, l’eqid de l'équipe et le nombre d'athlètes, ordonner par nombre
d’athlètes décroissant.
Résultats (296 lignes) : (164,25) ; (165,25) ; (166,25) ; ... ; (180,2) ; (181, 2) ; (182, 2)

In [ ]:
query="""


"""
execute(connection,query,show=True)

9. Pour chaque catégorie, la catégorie et le nombre d'épreuves.
Résultats (3 lignes) : (Femmes,43) ; (Mixte,6) ; (Hommes,49)

In [ ]:
query="""


"""
execute(connection,query,show=True)

10. Pour chaque sport, le nom du sport et le nombre d'épreuves, ordonner par nombre d'épreuves
décroissant.

Résultats (15 lignes) : (Patinage de vitesse,12) ; (Ski de fond,12) ; ... ;(Hockey sur glace,2)

In [ ]:
query="""


"""
execute(connection,query,show=True)

11. Pour chaque pays, le code du pays, le nombre de médailles en épreuve individuelle gagnées
et le nombre d'athlètes ayant gagnés au moins une médaille. Ordonner par nombre de
médailles décroissant. Aide : 2 tables seulement sont nécessaires.
Résultats (24 lignes) : (NOR, 24,19) ; (NED,22,15) ; ...

In [ ]:
query="""


"""
execute(connection,query,show=True)

12. Pour chaque pays et sport, le code du pays, le sid du sport, le nombre de médailles en
épreuve individuelle gagnées, le nombre d'athlètes ayant gagnés au moins une médaille,
ordonner d'abord par code pays, puis par nombre de médailles décroissant.
Résultats (84 lignes) : (AUS,12,2,2); (AUS,15,1,1);(AUT,13,9,7);(AUT,15,2,2);...

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Partitionnement avec « group by / having »

13. L’eqid de la ou des équipes qui sont composées :
a) d'exactement 10 athlètes. Résultat (1 ligne) : 226

In [ ]:
query="""


"""
execute(connection,query,show=True)

b) du plus d’athlètes pour ces JO.

Résultats (3 lignes) : 164 ; 165 ; 166

In [ ]:
query="""


"""
execute(connection,query,show=True)

14. Le nombre d'épreuves en individuel où il y a eu au moins 100 participants.
Résultat (1 ligne ) : 2

In [ ]:
query="""


"""
execute(connection,query,show=True)

15. Le nom des pays qui ont gagné au moins 20 médailles aux épreuves individuelles.
Résultats (3 lignes) : Pays-Bas ; États-Unis ; Norvège

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Division en SQL

16. Le sid des sports qui ont des épreuves dans toutes les catégories existantes.
Résultats (3 lignes) : 1 ; 6 ; 7

Principe : pour chaque épreuve, on compte le nombre de catégories, puis on regarde si il est
égal au nombre total de catégories d’épreuves.

In [ ]:
query="""


"""
execute(connection,query,show=True)

In [ ]:
query="""

"""
execute(connection,query,show=True)

17. Le nom des pays qui ont participé aux épreuves en individuel de tous les sports en

individuel. Résultats (3 lignes) : (Russie,12) ; (États-Unis,12) ; (Italie,12)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## Requêtes avec paramètres

In [ ]:
cursor=connection.cursor()
cursor.execute("SELECT * FROM athlete WHERE aid=(%s)", '1')
print(cursor.fetchall())


# Transactions

In [ ]:
connection.commit()

In [ ]:
resultat=execute(connection, "SELECT * FROM athlete where aid<12",True)

In [ ]:
delete_comment = "DELETE FROM athlete where aid<10"
execute(connection, delete_comment)

In [ ]:
execute(connection, "SELECT * FROM athlete where aid < 12",True)

In [ ]:
connection.rollback()

In [ ]:
execute(connection, "SELECT * FROM athlete where aid < 12",True)

# Fermer la connexion

In [ ]:
connection.commit() # implicit avec close
connection.close()